#Classification using 5 classifiers


##Exploring Marketing Data Set:

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a4304825715343a6642819258b4d85cf6fd3c235a0945cdff1ed775352939148
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# Je télécharge le fichier zip depuis l'URL
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
!unzip bank.zip

--2023-11-29 23:03:13--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank.zip’

bank.zip                [  <=>               ] 565.47K  2.00MB/s    in 0.3s    

2023-11-29 23:03:14 (2.00 MB/s) - ‘bank.zip’ saved [579043]

Archive:  bank.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, LinearSVC, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline
from pyspark.sql import Row
from IPython.display import display
from pyspark.sql.functions import col, format_string

In [ ]:
# Je crée une session Spark
spark=SparkSession.builder.appName('project').getOrCreate()

In [ ]:
df = spark.read.csv("bank-full.csv", sep=';', inferSchema=True, header=True)
df.show(10)

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [ ]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [ ]:
print("Total number of customers:", df.count())

Total number of customers: 45211


In [ ]:
df.groupBy("y").count().show()
print("Subscribed customers:", (df.filter(df.y == "yes").count()))
print("Not subscribed customers:", (df.filter(df.y == "no").count()))

+---+-----+
|  y|count|
+---+-----+
| no|39922|
|yes| 5289|
+---+-----+

Subscribed customers: 5289
Not subscribed customers: 39922


##Data preprocessing

In [ ]:
# Colonnes catégorielles et caractéristiques
categorical_cols = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]
feature_cols = ["age", "balance", "day", "duration", "campaign", "pdays", "previous"]

# Indexation des colonnes catégorielles
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") for col in categorical_cols]

# Encodage one-hot des colonnes indexées
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_encoded") for col in categorical_cols]

# Assemblage de toutes les colonnes en une seule colonne vectorielle
assembler = VectorAssembler(inputCols=feature_cols + [col + "_encoded" for col in categorical_cols], outputCol="features")

In [ ]:
# Définition du pipeline de prétraitement
preprocessing_pipeline = Pipeline(stages=indexers + encoders)

# Transformation des données
transformed_data = preprocessing_pipeline.fit(df).transform(df)
transformed_data = assembler.transform(transformed_data)

# Affichage du résultat
transformed_data.select("features","y").show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------------+---+
|features                                                                                                         |y  |
+-----------------------------------------------------------------------------------------------------------------+---+
|(51,[0,1,2,3,4,5,8,19,23,26,28,30,33,35,47],[58.0,2143.0,5.0,261.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|no |
|(51,[0,1,2,3,4,5,9,20,22,26,28,30,33,35,47],[44.0,29.0,5.0,151.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])  |no |
|(51,[0,1,2,3,4,5,14,19,22,26,28,31,33,35,47],[33.0,2.0,5.0,76.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])   |no |
|(51,[0,1,2,3,4,5,7,19,25,26,28,30,33,35,47],[47.0,1506.0,5.0,92.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |no |
|(51,[0,1,2,3,4,5,18,20,25,26,29,30,33,35,47],[33.0,1.0,5.0,198.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])  |no |
|(51,[0,1,2,3,4,5,8,19,23,26,28,30,33,35

In [ ]:
# Division en ensemble d'entraînement (80%) et de test (20%) avec seed pour la reproductibilité
(trainingData, testData) = transformed_data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Convertir la colonne y en numérique
label_indexer = StringIndexer(inputCol="y", outputCol="label")

## Compare all classifiers

In [ ]:
# Définition des modèles et de leurs noms
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", seed=42)
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed=42, numTrees=10)
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)
svm = LinearSVC(labelCol="label", featuresCol="features", maxIter=10)
gbt = GBTClassifier(labelCol="label", featuresCol="features", seed=42, maxIter=10)

models = [dt, rf, lr, svm, gbt]
model_names = ["Decision Tree", "Random Forest", "Logistic Regression", "Linear SVM", "GBT"]
model_metrics = []

**DecisionTreeClassifier:** Utilise un arbre de décision pour prendre des décisions en fonction des caractéristiques des données. Il divise récursivement les données en fonction de la meilleure caractéristique.

**RandomForestClassifier**: Construit un ensemble (forêt) d'arbres de décision et combine leurs prédictions pour améliorer la précision et la robustesse.

**Elle est souvent préférée dans les cas où la relation entre les variables indépendantes et dépendantes n'est pas linéaire**

**LogisticRegression**: Modélise la probabilité qu'une instance appartienne à une classe particulière à l'aide de la fonction logistique. Souvent utilisé pour des tâches de classification binaire.

**LinearSVC** (Support Vector Classifier linéaire) : Utilise des vecteurs de support pour définir un hyperplan qui sépare les classes de manière linéaire dans l'espace des caractéristiques.

**GBTClassifier** (Gradient Boosted Trees Classifier): Construit un ensemble de petits arbres de décision de manière séquentielle, où chaque arbre corrige les erreurs des précédents, améliorant ainsi la performance globale.

In [ ]:
# Entraînement et évaluation des modèles
for model, name in zip(models, model_names):
    pipeline = Pipeline(stages=[label_indexer, model])
    model_trained = pipeline.fit(trainingData)
    predictions = model_trained.transform(testData)
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

    # Matrice de confusion
    confusion_matrix = predictions.groupBy("label", "prediction").count()
    print(f"Confusion Matrix for {name}:")
    confusion_matrix.show()

    # Métriques
    metrics = {
        "Accuracy": round(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}),3),
        "Precision": round(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"}),3),
        "Recall": round(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"}),3),
        "F1-score": round(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}),3),
    }

    # Calcul de l'AUC
    auc_evaluator = BinaryClassificationEvaluator(labelCol="label")
    auc = auc_evaluator.evaluate(predictions)
    metrics["AUC"] = round(auc, 3)

    # Ajout des métriques dans une row
    metric_row = Row(model=name, **metrics)
    model_metrics.append(metric_row)

Confusion Matrix for Decision Tree:
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  308|
|  0.0|       1.0|  173|
|  1.0|       0.0|  700|
|  0.0|       0.0| 7850|
+-----+----------+-----+

Confusion Matrix for Random Forest:
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   67|
|  0.0|       1.0|   20|
|  1.0|       0.0|  941|
|  0.0|       0.0| 8003|
+-----+----------+-----+

Confusion Matrix for Logistic Regression:
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  353|
|  0.0|       1.0|  209|
|  1.0|       0.0|  655|
|  0.0|       0.0| 7814|
+-----+----------+-----+

Confusion Matrix for Linear SVM:
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  249|
|  0.0|       1.0|  139|
|  1.0|       0.0|  759|
|  0.0|       0.0| 7884|
+-----+----------+-----+

Confusion Matrix for GBT:
+-----+----------+-----

In [ ]:
# Création du DataFrame avec les métriques
metric_df = spark.createDataFrame(model_metrics)

# Affichage du DataFrame avec une largeur plus grande
display(metric_df.toPandas())

,model,Accuracy,Precision,Recall,F1-score,AUC
0,Decision Tree,0.903,0.887,0.903,0.888,0.524
1,Random Forest,0.894,0.881,0.894,0.852,0.879
2,Logistic Regression,0.904,0.890,0.904,0.892,0.912
3,Linear SVM,0.901,0.882,0.901,0.880,0.912
4,GBT,0.906,0.891,0.906,0.892,0.915


**Accuracy** (Exactitude) : Mesure la proportion d'observations correctement classées par le modèle, c'est-à-dire le nombre total de prédictions correctes divisé par le nombre total d'observations.

**Precision** (Précision) : Indique la proportion d'observations positives prédites correctement parmi toutes les observations positives prédites. C'est le nombre de vrais positifs divisé par la somme des vrais positifs et des faux positifs.

**Recall** (Rappel ou Sensibilité) : Représente la proportion d'observations positives prédites correctement parmi toutes les observations réellement positives. C'est le nombre de vrais positifs divisé par la somme des vrais positifs et des faux négatifs.

**F1-score**: Il s'agit d'une métrique qui combine la précision et le rappel en une seule valeur. C'est la moyenne harmonique de la précision et du rappel. Il est particulièrement utile lorsque les classes sont déséquilibrées.

**AUC** (Area Under the ROC Curve) : Mesure la capacité d'un modèle à discriminer entre les classes. La courbe ROC (Receiver Operating Characteristic) est un graphique qui représente le taux de vrais positifs par rapport au taux de faux positifs à différents seuils de classification. L'AUC mesure l'aire sous cette courbe, où une valeur de 1.0 indique une excellente capacité de discrimination.

##Conclusion

In [ ]:
def calculate_average_scores(model_metrics):
    # Calcul de la moyenne des metrics pour chaque modèle
    model_metrics_avg = model_metrics.withColumn("avg_score", (col("Accuracy") + col("Precision") + col("Recall") + col("F1-score") + col("AUC")) / 5)
    return model_metrics_avg

def best_algorithm(model_metrics):
    # Appeler la fonction pour calculer la moyenne
    model_metrics_avg = calculate_average_scores(model_metrics)

    # Trouver le modèle avec la meilleure moyenne
    best_model = model_metrics_avg.orderBy(col("avg_score").desc()).select("model").first()["model"]
    return best_model

def display_average_scores(model_metrics):
    # Appeler la fonction pour calculer la moyenne
    model_metrics_avg = calculate_average_scores(model_metrics)

    # Formater la colonne "avg_score" avec 4 décimales
    model_metrics_avg = model_metrics_avg.withColumn("avg_score", format_string("%.4f", col("avg_score")))

    # Sélectionner les colonnes nécessaires
    result_df = model_metrics_avg.select("model", "avg_score").toPandas()
    return result_df



# Utiliser la fonction et afficher le DataFrame Pandas
display(display_average_scores(metric_df))

# Trouver le meilleur algorithme
meilleur_algo = best_algorithm(metric_df)
print("\nL'algorithme le plus performant est :", meilleur_algo)


,model,avg_score
0,Decision Tree,0.8210
1,Random Forest,0.8800
2,Logistic Regression,0.9004
3,Linear SVM,0.8952
4,GBT,0.9020



L'algorithme le plus performant est : GBT


- RandomForestClassifier est préférable à DecisionTreeClassifier en raison de sa capacité à réduire le surajustement (overfitting) en agrégeant les prédictions de plusieurs arbres de décision, ce qui améliore généralement la généralisation du modèle.

-  AUC de Dec Tree indiquant une capacité limitée à discriminer entre les classes positives et négatives.

